In [115]:
import openpyxl as xl
import pprint as pp

In [116]:
#Load xlsx file into Python
wb = xl.load_workbook('Merged_List.xlsx') #Open Input workbook
journals = wb['journal_sheet'] #Open journals worksheet
urls = wb["url_sheet"]
num_rows = len(tuple(journals.rows))

#journal_sheet column map
#Col 0/A = journal name
#Col 1/B = Publisher
#Col 2/C = ISSN
#Col 3/D = EISSN
#Col 4/E = Country
#Col 5/F = Language
#Col 6/G = Category
#Col 7/H = Submission Guidelines URL



In [117]:
#Iterate through the url sheet to find the submission page URL outlines

access_methods = {}
publishers = []
url_methods = []

for row in urls.iter_rows(min_row=2, max_col=8, max_row=15):
    publishers.append(row[0].value)
    url_methods.append(row[7].value) 
    
access_methods = dict(zip(publishers,url_methods))

In [118]:
#Iterate through journals and replace None Values with empty strings
for row in journals.iter_rows(min_row=2, max_col=7, max_row=num_rows):
    for col in row:
        if col.value == None:
            col.value = ""


In [121]:
def get_url(row, source):
    #Alias varibles for readability
    
    journal = row[0]
    publisher = row[1]
    issn = row[2]
    eissn = row[3]
    target = row[7]
    
    #Check for each attribute in URL and replace with appropriate variable
    
    if "JNAME" in source:
        #Replace JNAME with lowercase journal name and replace space with hyphen
        temp_name = journal.value.lower().replace(" ", "-") 
        target = source.replace("JNAME",temp_name)
        source = target
    if "EISSN" in source:
        #Replace EISSN with actual EISSN, replacing space with hyphen
        temp_name = eissn.value.replace("-"," ")
        target = source.replace("EISSN", temp_name)
        source = target
    if "E_H_SSN" in source:
        #Replace EISSN with actual EISSN, keeping hyphens
        temp_name = eissn.value
        target = source.replace("E_H_SSN", temp_name)
        source = target
    if "ISSN" in source:
        #Replace ISSN with actual ISSN, replacing space with hyphen
        temp_name = issn.value.replace("-","")
        target = source.replace("ISSN", temp_name)
        source = target
    if "I_H_SSN" in source:
        #Replace ISSN with actual ISSN, keeping hyphens
        temp_name = issn.value
        target = source.replace("I_H_SSN", temp_name)
        source = target
    print(target)
    return target
    
    
    
    
    
    

In [122]:
#Iterate over all rows, find the URL outline and return the submission outline Page

for row in journals.iter_rows(max_col=8, min_row=2, max_row=num_rows):
    for key, value in access_methods.items():
            if key == row[1].value:
                try:
                    row[7].value = get_url(row, access_methods[row[1].value])#Pass row and url outline
                #journals['H'+str(row_count)].value = access_methods[row[1].value]
                except:
                    continue
            row_count += 1

https://www.elsevier.com/journals/brain-and-cognition/0278-2626/guide-for-authors
https://www.elsevier.com/journals/brain-and-language/0093-934X/guide-for-authors
https://www.elsevier.com/journals/cognitive-psychology/0010-0285/guide-for-authors
https://www.elsevier.com/journals/consciousness-and-cognition/1053-8100/guide-for-authors
https://www.elsevier.com/journals/contemporary-educational-psychology/0361-476X/guide-for-authors
https://www.elsevier.com/journals/developmental-review/0273-2297/guide-for-authors
https://www.elsevier.com/journals/explorations-in-economic-history/0014-4983/guide-for-authors
https://www.elsevier.com/journals/finance-research-letters/1544-6123/guide-for-authors
https://www.elsevier.com/journals/games-and-economic-behavior/0899-8256/guide-for-authors
https://www.elsevier.com/journals/historia-mathematica/0315-0860/guide-for-authors
https://www.elsevier.com/journals/journal-of-anthropological-archaeology/0278-4165/guide-for-authors
https://www.elsevier.com/jo

https://au.sagepub.com/en-gb/oce/journal/journal-of-language-and-social-psychology#submission-guidelines
https://au.sagepub.com/en-gb/oce/journal/journal-of-law-medicine-&-ethics#submission-guidelines
https://au.sagepub.com/en-gb/oce/journal/journal-of-leadership-&-organizational-studies#submission-guidelines
https://au.sagepub.com/en-gb/oce/journal/journal-of-learning-disabilities#submission-guidelines
https://au.sagepub.com/en-gb/oce/journal/journal-of-literacy-research#submission-guidelines
https://au.sagepub.com/en-gb/oce/journal/journal-of-macromarketing#submission-guidelines
https://au.sagepub.com/en-gb/oce/journal/journal-of-management#submission-guidelines
https://au.sagepub.com/en-gb/oce/journal/journal-of-management-inquiry#submission-guidelines
https://au.sagepub.com/en-gb/oce/journal/journal-of-mixed-methods-research#submission-guidelines
https://au.sagepub.com/en-gb/oce/journal/journal-of-palliative-care#submission-guidelines
https://au.sagepub.com/en-gb/oce/journal/journa

In [ ]:
wb.save('output.xlsx')